<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TDTitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
---
# Classification de chance de survie au Titanic

## Données du naufrage 
Le TD utilisera l'ensemble de données issues de 
[Titanic Dataset](https://www.kaggle.com/datasets/sakshisatre/titanic-dataset?select=The+Titanic+dataset.csv) que vous devez télécharger (il vous faudra créer un compte (gratuit)).

Ce fichier CSV contient presque 1300 données sur les voyageurs présents sur le Titanic. Pour chaque personne il y a: 
 - le nom, la classe, la survie,  le genre, l'âge, le nombre de personnes (si voyage en famille), le prix du ticket et le type de cabine.

---
Vous vous baserez sur les  solutions données à la création de réseaux de neurones.

- Vous utiliserez les colonnes pclass, survived, gender, age, family, fare, embarked.
  - la colonne survived étant la sortie.


----
Vous utiliserez **la bibliothèque Pandas** pour récupérer les valeurs.

In [ ]:
import pandas as pd

#################
#lire un fichier csv
filename="The Titanic dataset.csv"
df = pd.read_csv(filename)

#lire un fichier csv où les valeurs sont séparées par des tabulation sous un encodate ISO 8859-1
#df = pd.read_csv(filename, sep='\t', encoding='ISO-8859-1')


#################
#extraire les colonnes
col_class = df['pclass']

----
### Normalisation des données.

Il faut donner en entrée du réseau de neurone un vecteur de réels entre  0 et 1.
- Il faut donc normaliser les valeurs.
- Ainsi l'exemple 1 : 
  - [3, male, 42, 0, 7.55] devient [1, 1, 0.525,0,0.08] (3/3 (classe max), 42/80 (age max), 7.55/93 (prix max), male=1, female=0,embarked=0 si pas de cabine, 1 pour cabine S, 2 pour C, 3 pour Q)
  - la sortie de l'exemple 1 est [0]

---
### Classification
Créer le réseaux de neurones de votre choix.
Garder 0.2% des exemples comme valeurs de tests.
